# Análise de um Ativo
Análise exploratória e indicadores de retorno, risco e performance de um ativo financeiro.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Acoes = "ITUB4.SA"
Periodo_Inicio = datetime.datetime(2016,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
Dados = pdr.get_data_yahoo(Acoes, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados

### Análise descritiva dos Preços
- Contagem de dias.
- Média dos preços do ativo.
- Desvio padrão dos preços.
- Preço Mínimo.
- Preço Máximo.
- Quartis dos Preços.

In [ ]:
Dados.describe()

In [ ]:
SeriePrecos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados.Close))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

In [ ]:
CandlePrecos = plg.Figure(plg.Candlestick(x=Dados.index,
                                          open=Dados.Open,
                                          high=Dados.High,
                                          low=Dados.Low,
                                          close=Dados.Close))
CandlePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(CandlePrecos, "Preços")

### Cálculo e análise descritiva dos Retornos Diários
- Retorno Médio do Ativo (Média).
- Risco do Ativo (Desvio Padrão).
- Maior perda do período (Mínimo).
- Maior ganho no período (Máximo).
- Quartis do Risco.

In [ ]:
Dados["Retorno"] = Dados.Close.pct_change().fillna(0)

In [ ]:
Dados.Retorno.describe()

In [ ]:
Dados["RetornoLog"] = np.log(1+Dados.Retorno)

In [ ]:
Dados.RetornoLog.describe()

In [ ]:
SerieRetornos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados.Retorno))
SerieRetornos.update_layout(yaxis_tickformat=".1%")
Plot_Show(SerieRetornos, "Retornos")

In [ ]:
HistogramaRetornos = plg.Figure(plg.Histogram(x=Dados.Retorno))
HistogramaRetornos.update_layout(xaxis_tickformat=".1%")
Plot_Show(HistogramaRetornos, "Retornos")

### Cálculo do Retorno Acumulado
Calcula dia-a-dia o retorno acumulado da série.

In [ ]:
Dados["RetornoAcumulado"] = (1+Dados.Retorno).cumprod()-1

In [ ]:
Dados[["Close", "Retorno", "RetornoAcumulado"]]

In [ ]:
RetornoAcumulado = Dados.RetornoAcumulado[-1]
print("Retorno do Período: {0:.2%}".format( RetornoAcumulado ))

In [ ]:
SerieRetornosAcumulados = plg.Figure(plg.Scatter(x=Dados.index, y=Dados.RetornoAcumulado))
SerieRetornosAcumulados.update_layout(yaxis_tickformat=".1%")
Plot_Show(SerieRetornosAcumulados, "Retornos Acumulados")

### Indicadores do Ativo

- Retorno Médio

In [ ]:
RetornoMedio = Dados.Retorno.mean()
print("Retorno Médio: {0:.2%}".format(RetornoMedio))

- Retorno Acumulado

In [ ]:
RetornoAcumulado = Dados.RetornoAcumulado[-1]
print("Retorno Acumulado: {0:.2%}".format(RetornoAcumulado))

- Risco

In [ ]:
Risco = Dados.Retorno.std()
print("Risco: {0:.2%}".format(Risco))

- Performance (Sharpe)

In [ ]:
Sharpe = RetornoMedio / Risco
print("Performance Sharpe: {0:.2%}".format(Sharpe))

- Risco e Retorno Anualizado

In [ ]:
Periodos = Dados.Retorno.count()
RiscoAnualizado = Risco * np.sqrt(220)
RetornoDiario = ((1+RetornoAcumulado)**(1/Periodos))-1
RetornoAnualizado = ((1+RetornoDiario)**(220/1))-1
print("Risco Anualizado: {0:.2%}".format(RiscoAnualizado))
print("Retorno Anualizado: {0:.2%}".format(RetornoAnualizado))

- VaR Histórico

In [ ]:
VarHistorico = np.quantile(Dados.Retorno, 0.05)
VarHistoricoPeriodos = np.sqrt([1,7,14,21]) * VarHistorico

InvestimentoPrevisto = 10000
PerdaMaxima = InvestimentoPrevisto * VarHistoricoPeriodos

print("VaR 1 dia: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[0], PerdaMaxima[0]))
print("VaR 7 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[1], PerdaMaxima[1]))
print("VaR 14 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[2], PerdaMaxima[2]))
print("VaR 21 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[3], PerdaMaxima[3]))

- VaR Paramétrico

In [ ]:
VarParam = RetornoMedio - norm.ppf(0.95) * Risco
VarParamPeriodos = np.sqrt([1,7,14,21]) * VarParam

InvestimentoPrevisto = 10000
PerdaMaxima = InvestimentoPrevisto * VarParamPeriodos

print("VaR 1 dia: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[0], PerdaMaxima[0]))
print("VaR 7 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[1], PerdaMaxima[1]))
print("VaR 14 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[2], PerdaMaxima[2]))
print("VaR 21 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[3], PerdaMaxima[3]))